# PyVG: Data Science to predict Video Games sales
>Equipe: Alexis Terrasse, Henri-François Mole, Hsan Drissi, Stephane Lelievre
>
>Promo: DS_Oct21
---
## Scraping JV.com - Alexis
<img src='https://image.jeuxvideo.com/medias-md/145432/1454322894-3281-card.png' width=500></img>

### Importation des bibliothèques

In [ ]:
from bs4 import BeautifulSoup # Importation de BeautifulSoup
import requests # Importation de requests
from time import sleep # Importation de sleep
import datetime # Importation de datetime
from tqdm import tqdm
import numpy as np # Importation de Numpy sous l'alias np
import pandas as pd # Importation de Pandas sous l'alias pd
import os # Importation de os

import warnings # Import de warnings - gestion des avertissements
warnings.filterwarnings('ignore')

# Chargement de Selenium et paramètrage des arguments
from selenium import webdriver
options= webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver= webdriver.Chrome('C:\chromedriver.exe',options=options) # A adapter en fonction de l'emplacement de chromedriver.exe

### Définition de fonctions

In [ ]:
#####################################################################
###### Fonction de récupération des données des commentaires  #######
#####################################################################

def retrieve_user_reviews(url):
    
    month_fr= ['janv.','févr.','mars','avr.','mai','juin','juil','août','sept.','oct.','nov.','déc.']
    
    user_n=[] # On y stockera le nom de l'utilisateur de la critique
    user_d=[] # On y stockera la date de la critique
    user_s=[] # On y stockera le score de la critique
    user_c=[] # On y stockera le texte de la critique

    headermap = {"User-Agent": "Mac Firefox"};
    markup = requests.get(url, headers=headermap).text
    soup= BeautifulSoup(markup,"lxml")
    n_pages= int(len(soup.select('.bloc-liste-num-page .lien-jv'))/2 + 1) #nb de pages de reviews ( len/2 )
    
    n_reviews= int(soup.select('.nb-total-avis')[0].text[:-5])
    if n_reviews == 1: start_r= 1
    else : start_r= 2
        
    for n in soup.select('.bloc-pseudo-avis')[start_r:]:
        user_n.append(n.text.strip())
                
    for n in soup.select('.bloc-date-avis')[start_r:]:
        if 'Posté hier' in n.text:
            date=datetime.date.today()-datetime.timedelta(1)
            user_d.append(str(date.day)+' '+month_fr[date.month-1]+str(date.year))
        elif 'il y a' in n.text:
            date = datetime.datetime.today()
            user_d.append(str(date.day)+' '+month_fr[date.month-1]+str(date.year))
        else: 
            if len(n.text) > 27 :
                user_d.append(n.text.strip()[10:-8])
            else: 
                date= datetime.date.today()
                user_d.append(n.text[10: -8]+str(date.year))
                
    for n in soup.select('.note-avis strong')[start_r:]:
        user_s.append(n.text.strip())
        
    for n in soup.select('.text-enrichi-forum')[start_r:]:
        user_c.append(n.text.strip())      
        
    if n_pages > 1:
        for p in range(2, n_pages+1):
            url_p= url+'?p='+str(p)
            markup = requests.get(url_p, headers=headermap).text
            soup= BeautifulSoup(markup,"lxml")
            
            for n in soup.select('.text-user')[start_r:]:
                user_n.append(n.text)
                
            for n in soup.select('.bloc-date-avis')[start_r:]:
                if 'Posté hier' in n.text: 
                    date=datetime.date.today()-datetime.timedelta(1)
                    user_d.append(str(date.day)+' '+month_fr[date.month-1]+str(date.year))
                elif 'il y a' in n.text:
                    date = datetime.datetime.today()
                    user_d.append(str(date.day)+' '+month_fr[date.month-1]+str(date.year))
                else: 
                    if len(n.text) > 27 :
                        user_d.append(n.text.strip()[10:-8])
                    else: 
                        date= datetime.date.today()
                        user_d.append(n.text[10: -8]+str(date.year))
                    
            for n in soup.select('.note-avis strong')[start_r:]:
                user_s.append(n.text.strip())
                
            for n in soup.select('.text-enrichi-forum')[start_r:]:
                user_c.append(n.text.strip())
                
    # On retourne les listes contenant les noms, les dates, les scores et le commentaires des reviews            
    return([user_n,user_d,user_s,user_c]) 

#####################################################################
########## Fonction de récupération des données du jeu  #############
#####################################################################

def retrieve_game_infos(url):    
 
    game_p=[]
    users_reviews=[]

    headermap = {"User-Agent": "Mac Firefox"};
    markup = requests.get(url, headers=headermap).text
    soup= BeautifulSoup(markup,"lxml")
    
    if soup.select('.gameHeaderBanner__title') == []: 
        title= soup.select('.articleHeader__title')[0].text[6:] # On stocke le nom du jeu
        game_t=title.split(' :')[0]
    else: game_t= soup.select('.gameHeaderBanner__title')[0].text # On stocke le nom du jeu
    
    if soup.select('.gameHeaderBanner__platformLink') != []:
        for elt in soup.select('.gameHeaderBanner__platformLink'):
            game_p.append(elt.text.strip()) # On stock la/les plateforme(s)  
        if 'Tout support' in game_p : game_p.remove('Tout support') 
        if len(game_p) > 1: game_p= ', '.join(game_p)
        else : game_p= game_p[0]
    else: game_p=[]
    
    game_rscore= soup.select('.bloc-avis-testeur strong')[0].text # On stocke la note rédac
    
    if len(soup.select('.bloc-avis-lecteur strong')) == 0 : 
        game_uscore= np.nan  # NaN s'il n'y a pas de score lecteur
        len_usereview= 0
    else : 
        game_uscore= soup.select('.bloc-avis-lecteur strong')[0].text # On stocke le score lecteur
        
        driver= webdriver.Chrome('C:\chromedriver.exe',options=options)
        driver.get(url);
        sleep(3)
        element= driver.find_elements_by_xpath('/html/body/div[3]/div[4]/div/div[2]/div[3]/div/div[2]/div[2]/a[1]')
        attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', element[0])
        users_reviews= retrieve_user_reviews('https://www.jeuxvideo.com/'+attrs['href'])    
        len_usereview= len(users_reviews[0])
    
    if soup.select('.reviewText') != []:
        game_rreview= soup.select('.reviewText')[0].text # #review rédac
    else : game_rreview= np.nan
    
    # On retourne le titre, la/les plateformes, le score jv.com, le score utilsateur moyen, les critiques utilisateurs et le nb de critiques utilisateurs 
    return(game_t, game_p, game_rscore, game_uscore, users_reviews, len_usereview)


###############################################################
########## Fonction de parcours des tests de jeu  #############
###############################################################

def scraping_tests_page(end_p,start_p):    

    df= pd.DataFrame(columns= ['Name','Platform','Critic_Score','User_Score','N_usereviews'])
    df_reviews= pd.DataFrame(columns= ['Name', 'Platform','Date_crit', 'Name_crit','Score_crit','Comment_crit'])
    driver= webdriver.Chrome('C:\chromedriver.exe',options=options)
    
    npages= 935 # Nombre de pages de test 
    
    for p in tqdm(range(end_p,start_p-1,-1)) :

        name=[]
        platform=[]
        rscore=[]
        uscore=[]
        ureview=[]
        len_ureview=[]
        
        url= 'https://www.jeuxvideo.com/tests/?p='+str(p)
        #print(url)
        driver.get(url)
        elems = driver.find_elements_by_xpath("//a[@href]") 
        
        check= False        
        for elem in elems:

            if check == True :
                if ('https://www.jeuxvideo.com/tests/?p=' in elem.get_attribute("href")) or ('https://www.jeuxvideo.com/tests.htm' in elem.get_attribute("href")):
                    break
                else: 
                    if ('https://www.jeuxvideo.com/test/' in elem.get_attribute("href")) or ('https://www.jeuxvideo.com/articles/' in elem.get_attribute("href")):
                        #print(elem.get_attribute("href"))
                        game_t, game_p, game_rscore, game_uscore, users_reviews, len_usereview= retrieve_game_infos(elem.get_attribute("href"))
                        name.append(game_t)
                        platform.append(game_p)
                        rscore.append(game_rscore)
                        uscore.append(game_uscore)
                        ureview.append(users_reviews)
                        len_ureview.append(len_usereview)            
                                               
            if elem.get_attribute("href") == 'https://www.jeuxvideo.com/tests/titre/':
                check= True     

        columns = {
            'Name': name,
            'Platform': platform,
            'Critic_Score': rscore,
            'User_Score': uscore,
            'N_usereviews': len_ureview}  

        df_p= pd.DataFrame(columns)
        df= pd.concat([df,df_p])

 

        df_reviews_p= pd.DataFrame({}, columns= ['Name', 'Platform','Date_crit', 'Name_crit','Score_crit','Comment_crit'])
        
        k=0
        for i in range(0,len(name)):#len(name)
            for j in range(0,len_ureview[i]):
                df_reviews_p.loc[k]= [name[i], platform[i], ureview[i][1][j], ureview[i][0][j], ureview[i][2][j], ureview[i][3][j]]
                k+=1                    
            k+=1

        df_reviews= pd.concat([df_reviews, df_reviews_p])
        
    # Génere un .csv avec les données pour chaque jeux de la page examinée
    file_name='jvcom_'+str(start_p)+'_'+str(end_p)+'.csv'
    df.to_csv(file_name, sep=",", encoding='utf-8', index=False)
        
    # Génere un csv pour les critiques de chaque jeu de la page examinée  
    file_name= 'jvcom_review_'+str(start_p)+'_'+str(end_p)+'.csv'        
    df_reviews.to_csv(file_name, sep=",", encoding='utf-8', index=False)

In [20]:
#On parcours l'intégralité des pages à rebours
end=935
start= end-19
check= True
while end > 0:
    
    check_file= 'jvcom_review_'+str(start)+'_'+str(end)+'.csv'
    
    # Si le fichier csv n'existe déjà on pas commence le scraping
    if os.path.exists(check_file) == False:
        scraping_tests_page(end,start)
    
    if (start -19 <= 0 ) & check:
        end=start-1
        start=1
        check=False

    else :
        end-=20
        start=end-19
    

100%|██████████| 15/15 [55:11<00:00, 220.75s/it]
